In [1]:
from preprocessing import Preprocessor
from training import Trainer, Metrics
import torch.nn as nn
import torch.optim as optim
from models.first_CNN import firstCNN

In [2]:
years = [str(y) for y in range(2006, 2015)]
classes = ["detritus", "Leptocylindrus", "Chaetoceros", "Rhizosolenia", "Guinardia_delicatula", "Cerataulina", 
           "Cylindrotheca", "Skeletonema", "Dactyliosolen", "Thalassiosira", "Dinobryon", "Corethron", "Thalassionema", 
           "Ditylum", "pennate", "Prorocentrum", "Pseudonitzschia", "Tintinnid", "Guinardia_striata", "Phaeocystis"]
pp = Preprocessor(years, include_classes=classes, train_eg_per_class=500)


In [3]:
pp.create_datasets([0.6,0.2,0.2])

In [4]:
trainLoader = pp.get_loaders('train', 128)
validLoader = pp.get_loaders('validation', 128)

In [5]:
trainer = Trainer(epochs = 10, loss_fn = nn.MSELoss, optimizer = optim.SGD, lr = 0.01, momentum = 0.9, useCuda=True)

In [6]:
model = firstCNN()

In [ ]:
trainer.train(model, trainLoader, validLoader)

# - or -

#model = trainer.load_full_model(model, "./models/firstCNN-1.0.pth")

/userhome/30/smaghani/URFP/models/first_CNN.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)


Training Loss: 0.0475093349814415
Training Accuracy: 0.0505
Valid Accuracy: 0.058
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Training Loss: 0.047507788985967636
Training Accuracy: 0.0495
Valid Accuracy: 0.0585
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Training Loss: 0.04750674217939377
Training Accuracy: 0.050333333333333334
Valid Accuracy: 0.052
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Training Loss: 0.0475008487701416
Training Accuracy: 0.050666666666666665
Valid Accuracy: 0.0545
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Training Loss: 0.04750845953822136
Training Accuracy: 0.048
Valid Accuracy: 0.054
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Training Loss: 0.04751861095428467
Training Accuracy: 0.0495
Valid Accuracy: 0.0515
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Training Loss: 0.04748713597655296
Training Accuracy: 0.0495
Valid Accuracy: 0.0515
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Training Loss: 0.04749227687716484
Training Accuracy: 0.047
Valid Accuracy: 0.0515
~~~~~~~~~~~~~~~~~~~

In [ ]:
testLoader = pp.get_loaders('test', 128)
pred, target = trainer.test(model, testLoader)

In [ ]:
met = Metrics(target, pred)
met.accuracy()
met.recall()
met.f_score()
met.plot_CM()